In [1]:
#import required libraries
import cv2
import face_recognition as fr
import numpy as np

In [2]:
video_capture = cv2.VideoCapture(0)  #capturevideo from wepcam


In [3]:
#face file loading 
faces = [fr.load_image_file("pinthu.jpg"), fr.load_image_file("stevejobs.jpg"),fr.load_image_file("elonmusk.webp"), fr.load_image_file("billgates.jpg")]

#extract facial known encoding
known_face_encodings = [fr.face_encodings(faces[0])[0],fr.face_encodings(faces[1])[0],fr.face_encodings(faces[2])[0],fr.face_encodings(faces[3])[0],]

#label to images
known_face_names = ["Pinthushan", "Steve Jobs", "Elon Musk", "Bill Gates"]

In [ ]:
process_this_frame = True


#Analysing webcam video continuously
while True:
    ret, frame = video_capture.read(0)   #capture frame from webcam
    
    if process_this_frame:
        small_frame = cv2.resize(frame,(0,0), fx = 0.25 , fy = 0.25) # resize image frame size
        rgb_frame = np.ascontiguousarray(small_frame[:, :, ::-1]) # convert BGR to RGB
        face_locations = fr.face_locations(rgb_frame) # detect face location from RGB frame
        face_encodings = fr.face_encodings(rgb_frame,face_locations) # calculate Encodings
        
        face_names=[]

        for face_encoding in face_encodings:
            name = "Unknown"
            matches = fr.compare_faces(known_face_encodings,face_encoding) 
            face_distances = fr.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances) # find closest matching face 
            
            if matches[best_match_index]:
                name = known_face_names[best_match_index]         # override matches name

            face_names.append(name)
                
    process_this_frame = not process_this_frame
    
    # show rectangle frame and persions name
    for (top, right, bottom, left),name in zip(face_locations, face_names):
        # multiply box cordination
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        
        # known face rectangle and label decorations
        if name != "Unknown":
            cv2.rectangle(frame, (left,top), (right, bottom), (0,255,0),2)
        
            cv2.rectangle(frame,(left, bottom -35), (right,bottom), (0,255, 0), cv2.FILLED)
        
        # unknown face rectangle and label decorations
        else:
            cv2.rectangle(frame, (left,top), (right, bottom), (0,0,255),2)
        
            cv2.rectangle(frame,(left, bottom -35), (right,bottom), (0,0, 255), cv2.FILLED)
        
        cv2.putText(frame, name,(left + 6,bottom - 6),cv2.FONT_HERSHEY_DUPLEX , 1.0 , (255, 255, 255), 1)
            
        
    cv2.imshow("Face Recognition", frame)  #display frame
    
    if cv2.waitKey(1) & 0xFF == ('q'):
        break
        
video_capture.release()
cv2.destroyAllWindows()
        
        